# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries 
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
nltk.download(['punkt', 'wordnet', 'omw-1.4','stopwords'])

# sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegressionCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\younk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\younk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\younk\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\younk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database

def load_data():
    df = pd.read_sql_table('message_category', 'sqlite:///DisasterResponse.db')
    X = df['message']
    Y = df.iloc[:, 5:41]
    category_names = list(np.array(Y.columns))
    return X, Y, category_names

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text, url_place_holder_string = "urlplaceholder"):
    '''
    Function that tokenizes the text data. 

    Arguments:
        text - Text messages that need tokenizing
    Output:
        clean_tokens - List of tokens extracted from the original text
    '''
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the input text
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extracting word tokens from the input text
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    tokens = word_tokenize(text)
    
    # Remove stop words from token list in each column
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatizer to standardize words
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#load data and tokenize X text-message
X, y, category_names = load_data()

In [43]:
def model_pipeline1():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ]))

    ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    
    ])
    
    return pipeline

def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ]))

    ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    
    ])

    parameters = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}
    cv = GridSearchCV(pipeline, param_grid = parameters)

    return cv 


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state = 48)

In [7]:
def train_model():
    '''Train pipeline'''
    model = model_pipeline1()
    model.fit(X_train, y_train)
    return model.fit(X_train, y_train)

train_model()

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001A00D61F520>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

def test_model():
    model = train_model()
    y_pred = model.predict(X_test)

    print(classification_report(y_test.values, y_pred, target_names = y.columns.values))

test_model()

                        precision    recall  f1-score   support

               request       0.74      0.52      0.62      1461
                 offer       0.00      0.00      0.00        45
           aid_related       0.76      0.62      0.68      3624
          medical_help       0.59      0.28      0.38       723
      medical_products       0.61      0.33      0.43       426
     search_and_rescue       0.66      0.18      0.29       229
              security       0.21      0.06      0.09       160
              military       0.57      0.37      0.45       282
                 water       0.75      0.61      0.68       553
                  food       0.81      0.64      0.71       968
               shelter       0.79      0.51      0.62       760
              clothing       0.76      0.51      0.61       142
                 money       0.67      0.29      0.40       210
        missing_people       0.58      0.11      0.18       103
              refugees       0.58      

c:\Users\younk\anaconda3\envs\mynewenv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\younk\anaconda3\envs\mynewenv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
from sklearn.model_selection import GridSearchCV

parameters = {'clf__estimator__criterion': ["gini", "entropy"],     
              'clf__estimator__n_jobs':[-1]}
              
pipeline = model_pipeline1()

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Get the prediction values from the grid search cross validator
y_test = cv.predict(X_test)
y_train = cv.predict(X_train)

In [ ]:
# Classification report on test data
print(classification_report(y_test.values, y_train, target_names=y.columns.values))

In [ ]:
# Classification report on training data
print('\n',classification_report(y_train.values, y_train, target_names=y.columns.values))

### 8. Try improving the model further. 
* other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def train_second_model():
    '''Train pipeline'''
    model = model_pipeline2()
    model.fit(X_train, y_train)
    return model.fit(X_train, y_train)

train_second_model()

In [ ]:
# build model with the second pipeline (first verb extractor)

def test_second_model():
    model = train_model()
    y_pred = model.predict(X_test)

    print(classification_report(y_test.values, y_pred, target_names = y.columns.values))

test_second_model()

### 9. Export model as a pickle file

In [ ]:
import pickle 

def export_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, 'wb'))